In [3]:
import numpy as np
import matplotlib as plt
from PIL import Image
import cv2

# 读入图像并转为灰度图
left_path = './image_stitching/left_1.jpg'
right_path = './image_stitching/right_1.jpg'

left_image = cv2.imread(left_path)
right_image = cv2.imread(right_path)

left_gray_image = cv2.cvtColor(left_image, cv2.THRESH_BINARY)
right_gray_image = cv2.cvtColor(right_image, cv2.THRESH_BINARY)

# 创建SIFT对象
sift = cv2.SIFT_create()
# 求解特征点和特征向量
kpsA, dpA = sift.detectAndCompute(left_gray_image, None)
kpsB, dpB = sift.detectAndCompute(right_gray_image, None)

# 构造BFMacher对象
bf = cv2.BFMatcher()

# 用knnMatch方法进行关键点多点匹配
matches = bf.knnMatch(dpA, dpB, k=2)

# 去除不可靠匹配
# 1）新建一个空列表good_matches
good_matches = []
# 2）遍历matches，如果第一对点的欧氏距离小于k倍第二对点的欧氏距离则说明第一对点匹配可靠，将这对点的索引号追加到good_matches
for m in matches:
    if len(m) == 2 and m[0].distance < 0.4 * m[1].distance:
        good_matches.append((m[0].queryIdx, m[0].trainIdx))

# 寻得可靠匹配点并转换类型
# 1)将keypoint的pt属性（关键点坐标）赋值给kps
kps1 = np.float32([kp.pt for kp in kpsA])
kps2 = np.float32([kp.pt for kp in kpsB])

# 2)将根据good_matches匹配好的序号，将kps1和kps2对应排序
kps1 = np.float32([kps1[a[0]] for a in good_matches])
kps2 = np.float32([kps2[a[1]] for a in good_matches])

# 求解转换矩阵
M, status = cv2.findHomography(kps2, kps1, cv2.RANSAC, 4.0)  # M未变换矩阵

# 图像空间变换
result = cv2.warpPerspective(right_image, M, (left_image.shape[1] + right_image.shape[1], right_image.shape[0]))

# 图像拼接
# 将变换后的图像对应要拼接原图像的地方用原图像覆盖。
result[0:left_image.shape[0], 0:left_image.shape[1]] = left_image

# 显示结果
cv2.imshow("left", left_image)
cv2.imshow("right", right_image)
cv2.imshow("result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()










error: OpenCV(4.6.0) C:\b\abs_74oeeuevib\croots\recipe\opencv-suite_1664548340488\work\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
